### **RECOMMENDER SYSTEM**

#### Loading libraries

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import re

pd.set_option("display.max_columns", 50)
sns.set_theme(style="darkgrid", palette="deep", font_scale=1, rc={"figure.figsize": (12, 6)})

In [14]:
criteria = pd.read_csv("data/kriteriaVO.csv")
contract_award = pd.read_csv("data/contract_award_HI.csv")
contract_evaluation_criterion = pd.read_csv("data/contract_evaluation_criterionvvz_HI.csv")
contract = pd.read_csv("data/contractvvz_HI.csv")
evaluation = pd.read_csv("data/evaluation_criterionvvz_HI.csv")

#### Exploratory analysis

In [15]:
def get_to_know(df: pd.DataFrame, name: str) -> None:

    # Printing basic information about the datasets
    print(f"|DATASET: {name}|\n{40*'='}\nShape: {df.shape}\n{20*'='}\n\
NaNs:\n{5*'-'}\n{df.isna().sum()}\n{20*'='}\nDtypes:\n{7*'-'}\n{df.dtypes}")


    # List of datasets for analysis
dfs = [
    ("kriteriaVO", criteria),
    ("contract_award", contract_award),
    ("contract_evaluation_criterion", contract_evaluation_criterion),
    ("contract", contract),
    ("evaluation", evaluation)
]

# Cycle through the datasets and apply the function
for name, df in dfs:
    get_to_know(df=df,name=name)
    print(f"{40*'#'}\n")

|DATASET: kriteriaVO|
Shape: (3981, 1)
NaNs:
-----
criterion    0
dtype: int64
Dtypes:
-------
criterion    object
dtype: object
########################################

|DATASET: contract_award|
Shape: (259171, 5)
NaNs:
-----
id                      0
contract_number    175705
title               57096
description        234043
contract_id             0
dtype: int64
Dtypes:
-------
id                 object
contract_number    object
title              object
description        object
contract_id        object
dtype: object
########################################

|DATASET: contract_evaluation_criterion|
Shape: (41364, 5)
NaNs:
-----
id                 0
name           41364
weight             0
type               0
contract_id        0
dtype: int64
Dtypes:
-------
id              object
name           float64
weight           int64
type            object
contract_id     object
dtype: object
########################################

|DATASET: contract|
Shape: (135446, 11)
NaNs:
-----

In [16]:
for name, df in dfs:
    display(HTML(f"<h3>{name}</h3>"))
    display(df.head(n=5))

,criterion
0,Cena bez DPH
1,Cena s DPH
2,"Kritériá, ktoré sa uplatnia pri vyhodnocovaní ..."
3,Cena
4,Kritéria nákladov


,id,contract_number,title,description,contract_id
0,3ce0c0ff-acbb-425f-bdf3-2e4df8079790,NaN,Oprava bytových jader ve 2. NP v objektu B 30 ...,NaN,827fd5d9-b88c-4871-8714-692a04f32cf7
1,cb1543cb-7131-43a9-994e-8dce3948dfb2,NaN,Dům Chopin - rekonstrukce objektu - 2. etapa,NaN,875c9c26-1a3f-49bc-ae89-55b317a21b64
2,5fcf5846-69a7-4d74-9e85-384a793745a0,NaN,část 1 - IT vybavení,Předmětem plnění je dodávka IT vybavení do dvo...,14d79674-7702-4309-8a68-d2a8a508ff16
3,354c73e4-f7d3-4282-a8de-1f5e45b3b658,NaN,část 2 - nábytek,Předmětem plnění je dodávka nábytku do dvou uč...,14d79674-7702-4309-8a68-d2a8a508ff16
4,4d716fa0-6571-4732-93d3-a5f9f558e397,NaN,část 3 - pomůcky,Předmětem plnění je dodávka pomůcek do dvou uč...,14d79674-7702-4309-8a68-d2a8a508ff16


,id,name,weight,type,contract_id
0,0e381014-41cf-473e-adce-ba2763063d38,NaN,100,AC_LOWEST_PRICE,827fd5d9-b88c-4871-8714-692a04f32cf7
1,0a6d231d-6038-447e-b8cb-7e134f6173db,NaN,100,AC_LOWEST_PRICE,875c9c26-1a3f-49bc-ae89-55b317a21b64
2,28343e86-be38-418e-b367-14c83aa3cb5a,NaN,100,AC_LOWEST_PRICE,14d79674-7702-4309-8a68-d2a8a508ff16
3,f7833860-2c7a-4357-a6d8-60a9659d61ee,NaN,100,AC_LOWEST_PRICE,5adec230-a5ee-4aa6-8328-88bd820165d8
4,00a01e7f-fd9c-4d57-ac5a-24b40fd7f9d6,NaN,100,AC_LOWEST_PRICE,18dd1c93-3c8b-4b66-a63e-82a45c9bb748


,id,is_framework_agreement,subject_type,procedure_type,has_parts,title,description,eu_funded,cpv_base_main,estimated_cost_value,estimated_cost_currency
0,827fd5d9-b88c-4871-8714-692a04f32cf7,False,WORKS,PT_SIMPLIFIED_CONTRACT,False,Oprava bytových jader ve 2. NP v objektu B 30 ...,Kompletní oprava 15 bytových jader ve 2. NP bu...,False,45453000,5911731.00,NaN
1,875c9c26-1a3f-49bc-ae89-55b317a21b64,False,WORKS,PT_OPEN,False,Dům Chopin - rekonstrukce objektu - 2. etapa,Předmětem veřejné zakázky jsou stavební práce ...,True,45454100,51000000.00,NaN
2,14d79674-7702-4309-8a68-d2a8a508ff16,False,SUPPLIES,PT_SIMPLIFIED_CONTRACT,True,UČEBNA FYZIKY; REGISTRAČNÍ ČÍSLO PROJEKTU: CZ....,Předmětem plnění této veřejné zakázky je dodáv...,True,39160000,3303827.72,NaN
3,bb8b98ab-754c-4afa-8106-c153e13e7efc,False,SERVICES,PT_RESTRICTED,False,DATA_VPN PL_4357,Datové služby umožňující datovou komunikaci se...,NaN,64200000,NaN,NaN
4,d365ebe6-cde8-408c-856d-6b496e103f49,False,SERVICES,PT_OPEN,True,Uzavření rámcových dohod na těžbu motorovou pi...,Předmětem veřejné zakázky je uzavření rámcovýc...,NaN,77211400,NaN,NaN


,id,name,weight,type,contract_part_id
0,d88d8343-600e-473b-9641-fc6dc7d6f676,NaN,100,AC_PRICE,916de771-7fc4-4ac6-aa41-33d2ba0a95cf
1,0cc63c35-57cd-4e6b-a00c-b94efc935108,NaN,100,AC_PRICE,a13a097f-359e-4c8c-a80e-85e9e66dfa27
2,95a2f5bc-1937-4597-b94a-73435f06d3fe,NaN,100,AC_PRICE,22d65852-9ce7-4bc2-87fd-958d5ee5cb78
3,996c112c-57ec-49ae-b980-4d116bb0e21e,NaN,100,AC_PRICE,c1e84303-d390-4df7-926d-f62574c582fb
4,96c119e6-ebe5-400f-816c-ecbdffabcaf1,NaN,100,AC_PRICE,e5359f4b-ba61-4028-9784-2d01ba198bb2


In [17]:
for name, df in dfs:
    display(HTML(f"<h4>{name}</h4>"))
    print(df.columns)

Index(['criterion'], dtype='object')


Index(['id', 'contract_number', 'title', 'description', 'contract_id'], dtype='object')


Index(['id', 'name', 'weight', 'type', 'contract_id'], dtype='object')


Index(['id', 'is_framework_agreement', 'subject_type', 'procedure_type',
       'has_parts', 'title', 'description', 'eu_funded', 'cpv_base_main',
       'estimated_cost_value', 'estimated_cost_currency'],
      dtype='object')


Index(['id', 'name', 'weight', 'type', 'contract_part_id'], dtype='object')


#### Numerical attributes in datasets  

Dataset: contract  
- cpv_base_main, estimated_cost_value, estimated_cost_currency  

Dataset: evaluation
- weight *(needs to be cnoverted from object to int64)

In [18]:
# Attribute weight in evaluation dataset includes also characters like % (sadly)
# evaluation["weight"].loc[evaluation["weight"] == "25 %"]

In [19]:
# Joining all values in evaluation["weight"] into a single string
all_weights = " ".join(evaluation["weight"].dropna().astype(str))

# Finds all non-numeric characters in the string
non_numeric_chars = set(re.findall(r"[^\d]", all_weights))
print(non_numeric_chars)

{'.', 'ž', 'f', ')', 'e', 'A', 's', ':', 'B', 'o', 'z', 'd', ',', 'i', 'š', 'ě', 'á', 'p', 'ř', 'í', '+', 'b', 'u', 'l', 'j', 'a', 'c', 'h', 'V', 'N', 'P', '=', 't', '%', 'Z', 'ý', 'R', ' ', 'r', 'm', '-', 'k', '/', 'I', 'ů', 'é', 'n', 'D', 'č', 'v'}


In [20]:
# Removing all non-numeric characters, including spaces
evaluation["weight_clean"] = pd.to_numeric(
    evaluation["weight"]
    .astype(str)
    .str.strip()
    .str.replace(r"[^\d]", "", regex=True),
    errors="coerce"
)

In [21]:
# Those rows are problematic, because they contain non-numeric characters
problematic_rows = evaluation[evaluation["weight_clean"].isna()]
display(problematic_rows)

,id,name,weight,type,contract_part_id,weight_clean
1238,54967445-ddd1-4172-9009-66808b53228a,NaN,Pevná cena,AC_PRICE,bea6b77d-2cc7-4ef2-bda1-4ae53248a85e,NaN
1523,9734b188-df9a-492f-b2ee-f589a918acb0,1,t,AC_COST,4468414e-bb13-42e7-bdcb-be7836d59974,NaN
3016,354a2b3c-036e-4332-be5b-1776e34331ad,NaN,Pevná cena,AC_PRICE,a4c8d1f6-9b36-4b39-ae49-8896652a127e,NaN
3150,ed9416d1-0000-4ba1-a95e-ec4252f10882,1,tuna,AC_COST,1b54458d-de54-4067-a355-70e0d80b05d5,NaN
10522,7d7f7b43-7db4-4134-971d-500f2b785257,1,t,AC_COST,7200ff28-e639-457c-b70c-4de865f25e48,NaN
...,...,...,...,...,...,...
158051,4c2b00b7-5c25-4a20-8dd2-11366aab518e,plocha povrchu pro stojící cestující,dle vzorce kriteriální funkce,AC_QUALITY,af94246e-99a9-4e35-9eeb-969c0a2fde86,NaN
158052,f16203c2-c9a4-41a7-b6f1-33ad0c0969b5,NaN,dle vzorce kriteriální funkce,AC_PRICE,af94246e-99a9-4e35-9eeb-969c0a2fde86,NaN
158195,c6fff4e0-38ec-444b-812d-e3f0f30c7bde,počet míst k sezení,dle vzorce kriteriální funkce,AC_QUALITY,ef4c9b68-5c72-4226-9a0d-983532a27b38,NaN
158196,393554ee-c3f1-4038-aafb-18373118fe05,plocha povrchu pro stojící cestující,dle vzorce kriteriální funkce,AC_QUALITY,ef4c9b68-5c72-4226-9a0d-983532a27b38,NaN


In [22]:
# Dropping problematic rows from the evaluation dataset
evaluation = evaluation.dropna(subset=["weight_clean"]).copy()
evaluation.loc[:, "weight_clean"] = evaluation["weight_clean"].astype(int)